# Node connectivity information
This example demonstrates some functionality related to node connectivity, such as:
- Counting number of links connected to a node
- Coutning number of enabled links connected to a node
- Getting a list of upstream/downstream links connected to a node

In [1]:
import pandas as pd
from mikeplus import Database

In [2]:
db = Database("../tests/testdata/Db/Sirius/Sirius.sqlite")
db

Database<'Sirius.sqlite'>

In [3]:
# Get alias to msm_Node table for convenience
msm_node = db.tables.msm_Node
msm_node

msm_NodeTable<Nodes>

In [4]:
# Get all node MUIDs
muids = msm_node.get_muids()
muids

['Node_36',
 'Node_27',
 'Node_28',
 'Node_30',
 'Node_31',
 'Node_32',
 'Node_33',
 'Node_34',
 'Inflow to_WWTP_Basin',
 'PS_To_WWTP']

In [5]:
# Get number of links connected to Node_33
msm_node.get_number_of_links("Node_33")

2

In [6]:
# Get number of links connected to Node_33 that are enabled
msm_node.get_number_of_links_enabled("Node_33")

2

In [7]:
# Get link MUIDs where "Node_33" is the upstream/from node.
upstream_links = msm_node.get_upstream_links("Node_33")
upstream_links

['Link_36']

In [8]:
# Confirm that the MUIDs make sense with a query
db.tables.msm_Link.select(["FromNodeID", "ToNodeID"]).where("FromNodeID = 'Node_36'").to_pandas()

,FromNodeID,ToNodeID
Link_29,Node_36,Node_27


In [9]:
# Get link MUIDs where "Node_33" is the downstream/to node.
downstream_links = msm_node.get_downstream_links("Node_33")
downstream_links

['Link_35']

In [10]:
# Confirm that the MUIDs make sense with a query
db.tables.msm_Link.select(["FromNodeID", "ToNodeID"]).where("ToNodeID = 'Node_33'").to_pandas()

,FromNodeID,ToNodeID
Link_35,Node_32,Node_33


In [11]:
# Same as above, but for all nodes and visualized in a DataFrame
data = {
    muid: {
        "num_of_links": msm_node.get_number_of_links(muid),
        "num_of_links_enabled": msm_node.get_number_of_links_enabled(muid),
        "num_upstream_links": len(msm_node.get_upstream_links(muid)),
        "num_downstream_links": len(msm_node.get_downstream_links(muid)),
    }
    for muid in msm_node.get_muids()
}
df = pd.DataFrame(data).T
df

,num_of_links,num_of_links_enabled,num_upstream_links,num_downstream_links
Node_36,1,1,1,0
Node_27,2,2,1,1
Node_28,2,2,1,1
Node_30,3,2,1,1
Node_31,2,2,1,1
Node_32,2,2,1,1
Node_33,2,2,1,1
Node_34,2,2,1,1
Inflow to_WWTP_Basin,1,1,0,1
PS_To_WWTP,0,0,0,0


In [12]:
# Close the database
db.close()